<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=352e4ce8a9703fa43cf7accf8647dce3509af066ff475f93358b92c9d039cd69
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:


      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-20 11:04:31
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.16 L
Current:  1.45 C
-------------------
Today PnL: 29.26 K (0.2%)
Current PnL: -20.88 L (-13.68%)
CY Booked + Current PnL: -6.92 L (-4.54%)
-------------------
Total profit:  1.85 L
Total loss:  -22.74 L
-------------------
Total Booked + Current PnL: 20.06 L (15.97%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.80 L (59.21%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.45,-8.11,8.85,0.02,14195.0,-14159.0,160400.0,144.42,76.0,M-SC,1.84,234.0,-1.00,1.12,39.52,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.13,-14.19,16.65,0.09,14398.0,-14304.0,86473.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.61,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.51,10.23,8.59,19.69,18155.0,19608.0,211350.0,-0.53,57.0,X-MC,3.64,78.0,1.08,1.47,17.72,XY25,NTT,AC
35,ICICIGI,2252.93,0.16,9.10,10.17,20.19,19342.0,15865.0,190185.0,-14.60,59.0,X-MC,7.71,68.0,0.82,1.32,25.59,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,0.31,13.10,7.33,21.40,22005.0,34773.0,300199.0,4.93,63.0,X-LC,8.07,12.0,1.58,2.09,20.61,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,3.31,6.37,17.31,24.79,46375.0,16051.0,267907.0,1.42,64.0,X-SC,2.28,86.0,0.35,1.86,30.99,X40N,NTT,MISC
57,RAJESHEXPO,518.00,3.08,-62.39,166.09,0.07,85683.0,-85589.0,51588.0,1790.15,68.0,L-SC,2.16,268.0,-1.00,0.36,28.30,OX40N,NTT,JEWELLERY
0,5PAISA,593.00,2.53,-34.87,73.34,12.90,104373.0,-76182.0,142314.0,109.13,63.0,H-SC,15.68,161.0,-0.73,0.99,18.33,OX40N,NTT,FINANCE
37,IEX,219.00,2.14,-4.96,56.38,48.63,107141.0,-9920.0,190034.0,-36.71,51.0,H-SC,15.33,136.0,-0.09,1.32,6.02,XR,NTT,MISC
50,LTIM,7201.88,1.93,11.28,18.32,31.66,50181.0,27764.0,273915.0,6.87,71.0,H-LC,11.91,43.0,0.55,1.91,51.98,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.0,-2.52,-26.76,95.04,42.84,45247.0,-17398.0,47608.0,-52.07,40.0,X-SC,37.15,83.0,-0.38,0.33,3.16,XY24,NTT,MISC
39,INDIGOPNTS,1408.0,-2.45,-8.11,8.85,0.02,14195.0,-14159.0,160400.0,144.42,76.0,M-SC,1.84,234.0,-1.00,1.12,39.52,OX40N,NTT,PAINTS
81,VALIANTORG,838.0,-2.24,-43.42,207.81,74.15,192359.0,-71040.0,92565.0,-266.56,30.0,H-SC,13.31,149.0,-0.37,0.64,18.65,XR,NTT,CHEMICALS
14,CAMPUS,393.0,-2.17,-12.65,52.59,33.28,77069.0,-21229.0,146546.0,-29.66,31.0,M-SC,3.04,222.0,-0.28,1.02,13.58,XY24,NTT,FOOTWEAR
47,KANSAINER,340.0,-1.74,-19.70,41.31,13.47,89453.0,-53127.0,216540.0,-66.95,36.0,H-SC,1.74,158.0,-0.59,1.51,10.10,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.15,-1.90,111.95,107.91,162139.0,-2812.0,144832.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.04,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.45,-8.11,8.85,0.02,14195.0,-14159.0,160400.0,144.42,76.0,M-SC,1.84,234.0,-1.00,1.12,39.52,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.13,-14.19,16.65,0.09,14398.0,-14304.0,86473.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.61,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.74,-19.70,41.31,13.47,89453.0,-53127.0,216540.0,-66.95,36.0,H-SC,1.74,158.0,-0.59,1.51,10.10,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.64,-22.44,70.23,32.03,71809.0,-29587.0,102248.0,-16.04,51.0,H-SC,6.38,148.0,-0.41,0.71,14.53,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.14,1.21,70.45,72.51,107629.0,1829.0,152774.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.31,XR,NTT,IT
38,INDIAMART,4810.62,-0.29,3.40,99.92,106.72,127432.0,4198.0,127534.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.58,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.29,3.40,99.92,106.72,127432.0,4198.0,127534.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.58,AR,ATH,MISC
85,WIPRO,420.00,0.14,1.21,70.45,72.51,107629.0,1829.0,152774.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.31,XR,NTT,IT
25,FINCABLES,1641.55,-0.15,-1.90,111.95,107.91,162139.0,-2812.0,144832.0,-20.15,42.0,M-SC,10.11,216.0,-0.02,1.01,1.04,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.68,-1.24,37.34,35.63,77281.0,-2608.0,206966.0,-14.97,37.0,H-MC,5.03,119.0,-0.03,1.44,15.22,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.57,-4.22,89.93,81.92,98885.0,-4840.0,109958.0,-43.90,33.0,M-SC,6.98,236.0,-0.05,0.76,30.17,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.11,-14.40,66.09,42.18,156381.0,-39791.0,236619.0,-23.08,21.0,X-LC,0.04,3.0,-0.25,1.65,0.11,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.21,-34.59,106.71,35.20,89809.0,-44508.0,84162.0,10.32,25.0,X-SC,11.46,92.0,-0.50,0.59,0.21,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.49,-13.25,71.13,48.45,123722.0,-26568.0,173938.0,-19.44,30.0,X-MC,6.93,64.0,-0.21,1.21,0.30,X40N,ATH,FINANCE
46,JSWINFRA,342.00,0.07,-6.51,23.24,15.23,43531.0,-13035.0,187312.0,-19.93,31.0,X-MC,6.15,66.0,-0.30,1.30,23.65,X40N,NTT,REALTY
34,HONAUT,58357.33,0.30,-18.16,67.21,36.84,70369.0,-23238.0,104700.0,-26.34,31.0,X-SC,1.43,90.0,-0.33,0.73,7.19,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.11,-14.40,66.09,42.18,156381.0,-39791.0,236619.0,-23.08,21.0,X-LC,0.04,3.0,-0.25,1.65,0.11,XY24,NTT,AUTO
43,ITC,452.00,0.79,-0.62,11.12,10.44,22118.0,-1237.0,198901.0,-40.74,47.0,X-LC,0.93,5.0,-0.06,1.38,4.91,X40,NTT,FMCG
36,ICICIPRULI,790.00,0.07,1.85,28.59,30.97,51463.0,3267.0,180005.0,-22.06,52.0,X-MC,1.43,75.0,0.06,1.25,14.64,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.30,-18.16,67.21,36.84,70369.0,-23238.0,104700.0,-26.34,31.0,X-SC,1.43,90.0,-0.33,0.73,7.19,X40N,ATH,ELECTRICAL
66,SIEMENS,4671.50,-1.17,-13.71,45.46,25.51,72997.0,-25520.0,160575.0,-1.39,59.0,H-LC,1.77,49.0,-0.35,1.12,18.33,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.11,-14.40,66.09,42.18,156381.0,-39791.0,236619.0,-23.08,21.0,X-LC,0.04,3.0,-0.25,1.65,0.11,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.21,-34.59,106.71,35.20,89809.0,-44508.0,84162.0,10.32,25.0,X-SC,11.46,92.0,-0.50,0.59,0.21,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.49,-13.25,71.13,48.45,123722.0,-26568.0,173938.0,-19.44,30.0,X-MC,6.93,64.0,-0.21,1.21,0.30,X40N,ATH,FINANCE
54,PAGEIND,51605.07,0.32,-4.89,32.54,26.07,50678.0,-8000.0,155740.0,-29.94,34.0,X-MC,8.42,55.0,-0.16,1.08,0.32,X40,ATH,APPARELS
55,PGHH,17907.65,0.38,-3.24,38.24,33.76,74304.0,-6510.0,194310.0,-31.71,36.0,X-MC,2.86,57.0,-0.09,1.35,1.79,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.52,-26.76,95.04,42.84,45247.0,-17398.0,47608.0,-52.07,40.0,X-SC,37.15,83.0,-0.38,0.33,3.16,XY24,NTT,MISC
60,RELAXO,1176.00,0.31,-46.27,183.44,52.31,143085.0,-67159.0,78001.0,-42.79,34.0,X-SC,3.26,91.0,-0.47,0.54,4.41,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.21,-34.59,106.71,35.20,89809.0,-44508.0,84162.0,10.32,25.0,X-SC,11.46,92.0,-0.50,0.59,0.21,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.30,-18.16,67.21,36.84,70369.0,-23238.0,104700.0,-26.34,31.0,X-SC,1.43,90.0,-0.33,0.73,7.19,X40N,ATH,ELECTRICAL
52,MEDANTA,1486.00,0.15,0.26,23.75,24.07,26238.0,287.0,110474.0,-7.89,38.0,X-SC,4.67,89.0,0.01,0.77,19.93,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.31,-14.21,39.03,19.28,115843.0,-49151.0,296805.0,-25.24,64.0,X-LC,7.94,1.0,-0.42,2.06,9.72,X40,ATH,IT
41,INFY,2275.00,-0.07,4.84,47.73,54.87,156564.0,15132.0,328020.0,-18.52,59.0,X-LC,3.42,2.0,0.10,2.28,13.52,X40,BTT,IT
76,TMPV,600.00,0.11,-14.40,66.09,42.18,156381.0,-39791.0,236619.0,-23.08,21.0,X-LC,0.04,3.0,-0.25,1.65,0.11,XY24,NTT,AUTO
82,VBL,671.64,0.32,-7.89,47.29,35.67,137580.0,-24914.0,290928.0,-18.89,46.0,X-LC,6.00,4.0,-0.18,2.02,4.87,X40N,ATH,FMCG
43,ITC,452.00,0.79,-0.62,11.12,10.44,22118.0,-1237.0,198901.0,-40.74,47.0,X-LC,0.93,5.0,-0.06,1.38,4.91,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.18,-11.07,104.29,81.67,87002.0,-10387.0,83423.0,7505.56,58.0,L-SC,18.16,271.0,-0.12,0.58,61.55,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.39,-11.08,56.93,39.54,51209.0,-11209.0,89950.0,-30.50,72.0,M-SC,6.88,220.0,-0.22,0.63,26.84,AR,ATH,AUTO
51,MASFIN,398.61,-0.81,-4.62,27.96,22.05,26129.0,-4530.0,93450.0,-17.80,53.0,H-SC,7.16,164.0,-0.17,0.65,35.84,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.25,-21.85,57.43,23.03,80370.0,-39127.0,139944.0,53.51,43.0,H-SC,12.21,167.0,-0.49,0.97,38.85,XR,NTT,BANKS
80,VAIBHAVGBL,521.00,-1.17,-23.33,109.70,60.77,153718.0,-42649.0,140126.0,61.43,47.0,H-SC,11.09,159.0,-0.28,0.97,26.19,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.45,-8.11,8.85,0.02,14195.0,-14159.0,160400.0,144.42,76.0,M-SC,1.84,234.0,-1.00,1.12,39.52,OX40N,NTT,PAINTS
13,BSOFT,831.70,-0.66,-29.53,113.97,50.79,112965.0,-41535.0,99118.0,-9.55,59.0,H-SC,7.90,151.0,-0.37,0.69,15.41,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.72,0.41,20.45,20.95,51699.0,1038.0,252806.0,-0.63,77.0,X-LC,11.49,31.0,0.02,1.76,36.20,X40,ATH,PAINTS
80,VAIBHAVGBL,521.00,-1.17,-23.33,109.70,60.77,153718.0,-42649.0,140126.0,61.43,47.0,H-SC,11.09,159.0,-0.28,0.97,26.19,XR,NTT,JEWELLERY
68,SONACOMS,806.63,1.39,-11.08,56.93,39.54,51209.0,-11209.0,89950.0,-30.50,72.0,M-SC,6.88,220.0,-0.22,0.63,26.84,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.93
1,25,44.96
2,50,75.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.77
MC    29.14
LC    25.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.02
X40      20.99
X40N     11.65
XR        9.50
XY25      9.17
AR        9.16
OX40N     7.76
X200      1.91
SR        1.02
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.34
X-MC    22.66
X-LC    20.02
M-SC    12.13
X-SC     7.87
H-MC     4.76
H-LC     3.03
L-SC     1.43
M-MC     1.39
M-LC     1.06
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.85,-3.87,37.46
IT,13.70,-13.09,71.52
FINANCE,9.48,-13.77,61.85
MISC,7.53,-20.03,69.20
PAINTS,6.03,-6.76,22.62
ELECTRICAL,5.69,-10.61,49.81
INSURANCE,4.49,2.24,32.21
PHARMA,3.67,-0.37,33.22
AUTO,3.36,-19.12,70.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2987714.0,21
XR,1260691.0,13
AR,1258314.0,10
X40,984927.0,14
X40N,695034.0,9
OX40N,678197.0,10
XY25,323305.0,6
SR,275082.0,2
MH,66219.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3480456.0,25
M-SC,1314909.0,15
X-MC,1169854.0,16
X-LC,884361.0,11
X-SC,709805.0,8
H-MC,394258.0,3
L-SC,255727.0,3
H-LC,123178.0,2
M-LC,107629.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188122.0      6
           AR         854059.0      5
           XR         781459.0      7
M-SC       XY24       771286.0      6
X-MC       X40        423305.0      7
X-LC       X40        388182.0      5
X-MC       XY24       344198.0      3
H-SC       OX40N      315515.0      4
M-SC       OX40N      276999.0      5
X-SC       XY24       276536.0      3
H-SC       SR         275082.0      2
X-SC       X40N       259829.0      3
X-MC       X40N       232091.0      4
X-LC       XY24       224532.0      2
H-MC       AR         211218.0      2
X-LC       X40N       203114.0      2
H-MC       XY24       183040.0      1
X-SC       X40        173440.0      2
X-MC       XY25       170260.0      2
L-SC       XR         170044.0      2
M-SC       XR         146584.0      2
           AR         120040.0      2
M-LC       XR         107629.0      1
L-SC       OX40N       85683.0      1
H-LC       AR          72997.0      1
X-LC       XY25        68533.0      2
H-SC       MH          66219.0      1
L-MC       XR          54975.0      1
H-LC       X200        50181.0      1
M-MC       XY25        45478.0      1
L-LC       XY25        39034.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 74.0 seconds
